In [6]:
# Initial imports.
import pandas as pd
from path import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [7]:
# Loading data after database join of seasons/quarters file and Apple's stock file
# https://towardsdatascience.com/4-tricks-you-should-know-to-parse-date-columns-with-pandas-read-csv-27355bb2ad0e#:~:text=By%20default%2C%20date%20columns%20are%20parsed%20using%20the,a%20different%20date%20format%2C%20for%20example%2C%20YYYY-DD-MM%20HH%3AMM%3ASS%3A
file_path =Path("./AAPL_Mock_ML_Open.csv")
apple_df = pd.read_csv(file_path, parse_dates=['Date'])
apple_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,1981-02-11,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,Gain
1,1981-02-12,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,Loss
2,1981-02-13,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,Loss
3,1981-02-17,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,Gain
4,1981-02-18,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,Gain


In [8]:
# Convert date into an integer since the system was not taking it as a float so that the ML model can read the date - for now.
# https://www.geeksforgeeks.org/convert-floats-to-integers-in-a-pandas-dataframe/
apple_df['Date'] = apple_df['Date'].astype(int)

In [9]:
# Must remain in order to activate label_binarize
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_Open'] = le.fit_transform(apple_gain_loss_df['Gain_Loss_Open'])
apple_gain_loss_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,350697600000000000,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,0
1,350784000000000000,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,1
2,350870400000000000,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,1
3,351216000000000000,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,0
4,351302400000000000,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,0


In [10]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_Open']), classes=[1,0])
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_Open'] = binarized_gain_loss
apple_gain_loss_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,350697600000000000,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,1
1,350784000000000000,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,0
2,350870400000000000,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,0
3,351216000000000000,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,1
4,351302400000000000,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,1


In [11]:
# Perform binary encoding of Season and Quarter columns
apple_binary_encoded = pd.get_dummies(apple_gain_loss_df, columns=["Season", "Quarter"])
apple_binary_encoded.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open,Season_Spring,Season_Winter,Quarter_Q1,Quarter_Q2
0,350697600000000000,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,1,0,1,1,0
1,350784000000000000,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,0,0,1,1,0
2,350870400000000000,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,0,0,1,1,0
3,351216000000000000,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,1,0,1,1,0
4,351302400000000000,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,1,0,1,1,0


In [12]:
# If in doubt look up scikit learn label_binarize
from sklearn.preprocessing import label_binarize

binarized_gain_loss = label_binarize(y=list(apple_gain_loss_df['Gain_Loss_Open']), classes=[1,0])
apple_gain_loss_df = apple_df.copy()
apple_gain_loss_df['Gain_Loss_Open'] = binarized_gain_loss
apple_gain_loss_df.head()

,Date,Season,Quarter,Open,High,Low,Close,Adj Close,Volume,Open_24hr_Change,Gain_Loss_Open
0,350697600000000000,Winter,Q1,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0.118304,0
1,350784000000000000,Winter,Q1,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,-0.001116,1
2,350870400000000000,Winter,Q1,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,-0.002233,1
3,351216000000000000,Winter,Q1,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0.001674,0
4,351302400000000000,Winter,Q1,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0.005023,0


In [13]:
# Define features set
X = apple_binary_encoded.copy()

In [14]:
y = apple_gain_loss_df["Gain_Loss_Open"]
X = X.drop(["Gain_Loss_Open","Open_24hr_Change"], axis=1)
X.head()

,Date,Open,High,Low,Close,Adj Close,Volume,Season_Spring,Season_Winter,Quarter_Q1,Quarter_Q2
0,350697600000000000,0.118304,0.118304,0.117746,0.117746,0.092428,13843200,0,1,1,0
1,350784000000000000,0.117188,0.117188,0.116629,0.116629,0.091552,14560000,0,1,1,0
2,350870400000000000,0.114955,0.114955,0.113839,0.113839,0.089362,11155200,0,1,1,0
3,351216000000000000,0.116629,0.117188,0.116629,0.116629,0.091552,12275200,0,1,1,0
4,351302400000000000,0.121652,0.122768,0.121652,0.121652,0.095495,19241600,0,1,1,0


In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)
X_train.shape

(30, 11)

In [16]:
#Create a Logistic Regression Model

from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs',
                                max_iter=200,
                                random_state=1)

In [17]:
#Fit (train) or model using the training data
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [18]:
# Make ppredictions
y_pred = classifier.predict(X_test)
results = pd.DataFrame({"Prediction": y_pred, "Actual": y_test}).reset_index(drop=True)
results.head(20)

,Prediction,Actual
0,0,0
1,0,0
2,0,1
3,0,0
4,0,1
5,0,0
6,0,1
7,0,1
8,0,1
9,0,0


In [19]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.45454545454545453
